# Salary prediction notebook

### Introduction, objectives etc

In [1]:
import pandas as pd
from scipy.stats import zscore

In [2]:
df = pd.read_csv("Salary_Data.csv")
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


# EDA

## Univariate analysis

 Each category

- missing values?
- outliers?
- how many categories for "education"
- how mnay categories for "job title"
- distribution of years of experience? 

In [3]:
df.isna().sum()

Age                    2
Gender                 2
Education Level        3
Job Title              2
Years of Experience    3
Salary                 5
dtype: int64

In [4]:
df.describe()


,Age,Years of Experience,Salary
count,6702.000000,6701.000000,6699.000000
mean,33.620859,8.094687,115326.964771
std,7.614633,6.059003,52786.183911
min,21.000000,0.000000,350.000000
25%,28.000000,3.000000,70000.000000
50%,32.000000,7.000000,115000.000000
75%,38.000000,12.000000,160000.000000
max,62.000000,34.000000,250000.000000


In [5]:
df['Age'].describe()

count    6702.000000
mean       33.620859
std         7.614633
min        21.000000
25%        28.000000
50%        32.000000
75%        38.000000
max        62.000000
Name: Age, dtype: float64

In [6]:
features_considered = ['Age', 'Years of Experience', 'Salary']
z_scores = zscore(df[features_considered], nan_policy = 'omit')
z_scores.head()

,Age,Years of Experience,Salary
0,-0.212877,-0.510797,-0.479839
1,-0.738221,-0.840909,-0.953483
2,1.494490,1.139763,0.656907
3,0.312467,-0.180685,-1.048212
4,2.413841,1.965043,1.604195


# Bivariate analysis

- Correlation between years of experience and salary
- scatterplots
- boxplots (all variables against salary and see if median is similar) 
    to understand what factors really affect salary
- categorical variables and their relationships
- how variables relate to salary

In [7]:
df["Job Title"].value_counts

<bound method IndexOpsMixin.value_counts of 0           Software Engineer
1                Data Analyst
2              Senior Manager
3             Sales Associate
4                    Director
                ...          
6699    Director of Marketing
6700          Sales Associate
6701        Financial Manager
6702        Marketing Manager
6703          Sales Executive
Name: Job Title, Length: 6704, dtype: object>

In [8]:
unique_values = df['Job Title'].unique()

print(unique_values)

['Software Engineer' 'Data Analyst' 'Senior Manager' 'Sales Associate'
 'Director' 'Marketing Analyst' 'Product Manager' 'Sales Manager'
 'Marketing Coordinator' 'Senior Scientist' 'Software Developer'
 'HR Manager' 'Financial Analyst' 'Project Manager' 'Customer Service Rep'
 'Operations Manager' 'Marketing Manager' 'Senior Engineer'
 'Data Entry Clerk' 'Sales Director' 'Business Analyst' 'VP of Operations'
 'IT Support' 'Recruiter' 'Financial Manager' 'Social Media Specialist'
 'Software Manager' 'Junior Developer' 'Senior Consultant'
 'Product Designer' 'CEO' 'Accountant' 'Data Scientist'
 'Marketing Specialist' 'Technical Writer' 'HR Generalist'
 'Project Engineer' 'Customer Success Rep' 'Sales Executive' 'UX Designer'
 'Operations Director' 'Network Engineer' 'Administrative Assistant'
 'Strategy Consultant' 'Copywriter' 'Account Manager'
 'Director of Marketing' 'Help Desk Analyst' 'Customer Service Manager'
 'Business Intelligence Analyst' 'Event Coordinator' 'VP of Finance'
 'G

In [9]:
to_read = open("Non_CS_Jobs.txt")
Non_CS_Jobs = []
while(True):
    content = to_read.readline()
    if not content:
        break
    content = content.strip()
    Non_CS_Jobs.append(content[1: (len(content) - 1)])

    print(content[1: (len(content) - 1)])


Sales Associate
Director
Marketing Analyst
Product Manager
Sales Manager
Marketing Coordinator
Senior Scientist
HR Manager
Financial Analyst
Project Manager
Customer Service Rep
Operations Manager
Marketing Manager
Senior Engineer
Sales Director
Business Analyst
VP of Operation
Recruiter
Financial Manager
Social Media Specialist
Senior Consultant
Product Designer
CEO
Accountant
Marketing Specialist
Technical Writer
HR Generalist
Project Engineer
Customer Success Rep
Sales Executive
Operations Director
Administrative Assistant
Strategy Consultant
Copywriter
Account Manager
Director of Marketing
Customer Service Manager
Business Intelligence Analyst
Event Coordinator
VP of Finance
Graphic Designer
Social Media Manager
Director of Operations
Junior Accountant
Digital Marketing Manager
Customer Service Representative
Business Development Manager
Senior Financial Analyst
Research Director
Creative Director
Human Resources Director
Content Marketing Manager
Technical Recruiter
Sales Represe

Filtering out non CS jobs 

In [10]:
print(len(df.index))
mask = ~df['Job Title'].isin(Non_CS_Jobs)

# Filter the DataFrame using the mask
filtered_df = df[mask]



print(len(filtered_df.index))
filtered_df.head


6704
3645


<bound method NDFrame.head of        Age  Gender    Education Level           Job Title  \
0     32.0    Male         Bachelor's   Software Engineer   
1     28.0  Female           Master's        Data Analyst   
2     45.0    Male                PhD      Senior Manager   
10    29.0    Male           Master's  Software Developer   
18    25.0  Female         Bachelor's    Data Entry Clerk   
...    ...     ...                ...                 ...   
6230  32.0  Female                PhD      Data Scientist   
6233  27.0    Male  Bachelor's Degree   Software Engineer   
6244  32.0  Female                PhD      Data Scientist   
6247  27.0    Male  Bachelor's Degree   Software Engineer   
6258  32.0  Female                PhD      Data Scientist   

      Years of Experience    Salary  
0                     5.0   90000.0  
1                     3.0   65000.0  
2                    15.0  150000.0  
10                    3.0   75000.0  
18                    0.0   35000.0  
...      

Filtering out null values

In [11]:
print(len(filtered_df.index))
filtered_df = filtered_df.dropna(subset = ['Gender']).reset_index(drop = True)
filtered_df = filtered_df.dropna(subset = ['Education Level']).reset_index(drop = True)
filtered_df = filtered_df.dropna(subset = ['Job Title']).reset_index(drop = True)
filtered_df = filtered_df.dropna(subset = ['Salary']).reset_index(drop = True)

filtered_df.isna().sum()[filtered_df.isna().sum()>0]

print(len(filtered_df.index))
filtered_df.isna().sum()

#there are 0 negative salaries in the dataset, or all salaries are valid
condition = filtered_df['Salary'] < 0
count = condition.sum()
print(count)


3645
3641
0


Checking Years of Experience

In [12]:
years_condition = filtered_df['Years of Experience'] < 0
years_count = years_condition.sum()
print(years_count)


0


Checking Age

In [13]:
age_condition = filtered_df['Age'] < 0
age_count = age_condition.sum()
print(age_count)

0


In [14]:
filtered_df["Education Level"].value_counts

<bound method IndexOpsMixin.value_counts of 0              Bachelor's
1                Master's
2                     PhD
3                Master's
4              Bachelor's
              ...        
3636                  PhD
3637    Bachelor's Degree
3638                  PhD
3639    Bachelor's Degree
3640                  PhD
Name: Education Level, Length: 3641, dtype: object>

Making Education Levels uniform 

In [15]:
edu_unique_values = filtered_df['Education Level'].unique()

print(edu_unique_values)

filtered_df['Education Level'].replace('Bachelor\'s Degree', 'Bachelor\'s', inplace=True)
filtered_df['Education Level'].replace('Master\'s Degree', 'Master\'s', inplace=True)
filtered_df['Education Level'].replace('phD', 'PhD', inplace=True)
edu_unique_values = filtered_df['Education Level'].unique()

print(edu_unique_values)

["Bachelor's" "Master's" 'PhD' "Bachelor's Degree" "Master's Degree"
 'High School']
["Bachelor's" "Master's" 'PhD' 'High School']


In [16]:
filtered_df["Gender"].value_counts
gen_unique_values = filtered_df['Gender'].unique()

print(gen_unique_values)

['Male' 'Female' 'Other']


# Data preprocessing

- Treat out null values
- Treat outliers
- one-hot encode our categorical columns (gender, education, job title)
- 

Dropping Duplicates
Do we need to drop duplicates? There seems to be many (could be different people with identical data?)

In [17]:
filtered_df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,29.0,Male,Master's,Software Developer,3.0,75000.0
4,25.0,Female,Bachelor's,Data Entry Clerk,0.0,35000.0
...,...,...,...,...,...,...
3636,32.0,Female,PhD,Data Scientist,9.0,145000.0
3637,27.0,Male,Bachelor's,Software Engineer,3.0,80000.0
3638,32.0,Female,PhD,Data Scientist,9.0,145000.0
3639,27.0,Male,Bachelor's,Software Engineer,3.0,80000.0


In [18]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Now print the entire DataFrame
#print(filtered_df)

       Age  Gender Education Level                     Job Title  \
0     32.0    Male      Bachelor's             Software Engineer   
1     28.0  Female        Master's                  Data Analyst   
2     45.0    Male             PhD                Senior Manager   
3     29.0    Male        Master's            Software Developer   
4     25.0  Female      Bachelor's              Data Entry Clerk   
5     47.0    Male        Master's              VP of Operations   
6     30.0    Male      Bachelor's                    IT Support   
7     37.0  Female        Master's              Software Manager   
8     24.0    Male      Bachelor's              Junior Developer   
9     29.0    Male        Master's                Data Scientist   
10    46.0    Male             PhD                Senior Manager   
11    34.0  Female        Master's                   UX Designer   
12    30.0    Male      Bachelor's              Network Engineer   
13    25.0    Male      Bachelor's             H

Is 55 duplicates enough to warrant filtering out duplicates?

In [24]:
print(len(filtered_df.index))
indices = []
mydict = {}
for index, row in filtered_df.iterrows():
    a = [row['Age'], row['Gender'], row['Education Level'], row['Years of Experience'], row['Salary']]
    a = tuple(a)
    if a in mydict:
        mydict[a] += 1
        #print(a)
        indices.append(index)
        print("HELLO")
    else:
        mydict[a] = 1
print(len(indices))
for i in indices:
    filtered_df.drop(i, inplace = True)

sum = 0
for i in mydict.values():
    if i > 1:
        sum += i - 1

print(len(indices))
print(len(filtered_df.index))

896
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE
BYE


In [20]:

print("Number of rows before removing duplicates", len(filtered_df))
column_names = filtered_df.columns.tolist()
filtered_df = filtered_df.drop_duplicates()
print("Number of rows after removing duplicates", len(filtered_df))
filtered_df.head



Number of rows before removing duplicates 896
Number of rows after removing duplicates 896


<bound method NDFrame.head of        Age  Gender Education Level                     Job Title  \
0     32.0    Male      Bachelor's             Software Engineer   
1     28.0  Female        Master's                  Data Analyst   
2     45.0    Male             PhD                Senior Manager   
3     29.0    Male        Master's            Software Developer   
4     25.0  Female      Bachelor's              Data Entry Clerk   
5     47.0    Male        Master's              VP of Operations   
6     30.0    Male      Bachelor's                    IT Support   
7     37.0  Female        Master's              Software Manager   
8     24.0    Male      Bachelor's              Junior Developer   
10    46.0    Male             PhD                Senior Manager   
11    34.0  Female        Master's                   UX Designer   
12    30.0    Male      Bachelor's              Network Engineer   
13    25.0    Male      Bachelor's             Help Desk Analyst   
14    27.0  Female